In [1]:
import numpy as np
import tensorflow as tf
import chess
import pandas as pd
import io

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input 
from tensorflow.keras.models import load_model

from keras import backend as K
from tensorflow import keras

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


Helper Functions~~~

In [0]:
NUM_OUTPUTS = 5


def get_mat(board, piece, color):
  squareSet = board.pieces(piece, color)
  return get_bin_mat(squareSet);

def get_bin_mat(squareSet):
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in squareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
            
  return mat


def get_dual_bin_mat(piece):
  whiteSquareSet = board.pieces(piece, chess.WHITE)
  blackSquareSet = board.pieces(piece, chess.BLACK)
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in whiteSquareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
        elif(y*8 + x in blackSquareSet):
          mat[7 - y, x] = -1
            
  return mat


def interpret_pred(p):
  
#   moves = []
#   for out in p:
#     # There's an extra list wrapping for some reason
#     out = out[0]
#     start = np.argmax(out[:64])
#     end = np.argmax(out[64:])
       
#     moves.append((chess.SQUARE_NAMES[start],
#                   chess.SQUARE_NAMES[end]))
    
#   return moves

  moves = []
  for i,k in zip(p[0::2], p[1::2]):
    i = i[0]
    k = k[0]
    
    start = np.argmax(i)
    end = np.argmax(k)
       
    moves.append((chess.SQUARE_NAMES[start],
                  chess.SQUARE_NAMES[end]))
  
  return moves
  

def create_input(board):
  if(board.turn):
    turnMat = np.ones((8,8))
  else:
    turnMat = np.negative(np.ones((8,8)))
  mat = np.array(([get_mat(board, i, chess.WHITE) for i in range(1,7)] + [get_mat(board, i, chess.BLACK) for i in range(1,7)] + [turnMat]))
  y = mat.reshape((-1, 13, 8, 8))
  return y

def create_output(moves):
  output = []
  for i in range(0, NUM_OUTPUTS):
    startPos = moves[i][0]
    endPos = moves[i][1]
    
    startMat = np.zeros(64)
    endMat = np.zeros(64)
    
    startMat[startPos] = 1
    endMat[endPos] = 1
        
    output.append(startMat)
    output.append(endMat)
  
  return output

def create_training_set(df):
  inputs = []
  outputs = []

  for i in range(len(df)):
    pos = df.iloc[i].Position
    moves = eval(df.iloc[i].Moves)

    board.set_fen(pos)
    if i > 0:
      inputs = np.append(inputs, create_input(board), axis=0)
    else:
      inputs = create_input(board)
    #outputs.append(create_output(moves))
    
  for j in range(0, NUM_OUTPUTS):
    startOutput = []
    endOutput = []
    
    for k in range(len(df)):
      moves = eval(df.iloc[k].Moves)

      startPos = moves[j][0]
      endPos = moves[j][1]
    
      startMat = np.zeros(64)
      endMat = np.zeros(64)

      startMat[startPos] = 1
      endMat[endPos] = 1

      startOutput.append(startMat)
      endOutput.append(endMat)
    
    outputs.append(startOutput)
    outputs.append(endOutput)

  return (inputs, outputs)

Input data:

  (6 pieces) * 2 players + 1 for turn
  
  8 rows
  
  8 columns
  
 => 8 * 8 * 13 = 832
 
 
 Output data:
 
 - Assume predicting five moves
 - initial location, and new location
 
 
Array of objects/tuples
[{initial location, new location},...]
 
 
 => 5 * 2 * 64 = 640
  

In [3]:
inp = Input((13,8,8))


x = Conv2D(250, kernel_size=3, activation='relu', input_shape=(13,8,8))(inp)

x = Conv2D(250, kernel_size=3, activation='relu', input_shape=(13,8,8))(x)

x = Conv2D(250, kernel_size=3, activation='relu', input_shape=(13,8,8))(x)


x = Flatten()(x)


outputs = []

for n in range(NUM_OUTPUTS * 2):
  outputs.append(Dense(64, activation='softmax')(x))

model = Model(inp, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13, 8, 8)]   0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 11, 6, 250)   18250       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 9, 4, 250)    562750      conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 2, 250)    562750      conv2d_1[0

In [4]:
board = chess.Board()

y = create_input(board)

p = model.predict(y)

interpret_pred(p)

[('e4', 'd7'), ('b8', 'c7'), ('f1', 'a8'), ('b4', 'e6'), ('g8', 'g5')]

In [5]:
df=pd.read_csv('drive/My Drive/Chess/train2.csv')

df.head()

,Position,Moves
0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,"[(4, 28), (4, 28), (4, 28), (4, 28), (4, 28)]"
1,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,"[(4, 28), (4, 28), (4, 28), (4, 28), (4, 28)]"
2,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,"[(4, 28), (4, 28), (4, 28), (4, 28), (4, 28)]"
3,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,"[(4, 28), (4, 28), (4, 28), (4, 28), (4, 28)]"
4,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,"[(4, 28), (4, 28), (4, 28), (4, 28), (4, 28)]"


In [6]:
df = df[:1000] # Set an initial limit

SPLIT_SIZE = 1000
NUM_CHUNKS = len(df)/SPLIT_SIZE

i = 1
for chunk in np.array_split(df, NUM_CHUNKS):
  inputs, outputs = create_training_set(chunk)
  model.fit(inputs, outputs, epochs=5)
  print('Model fit for ' + str(i * SPLIT_SIZE) + 'training samples.')
  i = i + 1

Epoch 1/5
1000/1000 [==============================] - 2s 2ms/sample - loss: 5.2222 - dense_loss: 0.5258 - dense_1_loss: 0.5177 - dense_2_loss: 0.5090 - dense_3_loss: 0.5130 - dense_4_loss: 0.5080 - dense_5_loss: 0.5072 - dense_6_loss: 0.5104 - dense_7_loss: 0.5041 - dense_8_loss: 0.5050 - dense_9_loss: 0.4995 - dense_acc: 0.9680 - dense_1_acc: 0.9680 - dense_2_acc: 0.9680 - dense_3_acc: 0.9680 - dense_4_acc: 0.9680 - dense_5_acc: 0.9680 - dense_6_acc: 0.9680 - dense_7_acc: 0.9680 - dense_8_acc: 0.9680 - dense_9_acc: 0.9680
Epoch 2/5
1000/1000 [==============================] - 1s 805us/sample - loss: 0.0000e+00 - dense_loss: 0.0000e+00 - dense_1_loss: 0.0000e+00 - dense_2_loss: 0.0000e+00 - dense_3_loss: 0.0000e+00 - dense_4_loss: 0.0000e+00 - dense_5_loss: 0.0000e+00 - dense_6_loss: 0.0000e+00 - dense_7_loss: 0.0000e+00 - dense_8_loss: 0.0000e+00 - dense_9_loss: 0.0000e+00 - dense_acc: 1.0000 - dense_1_acc: 1.0000 - dense_2_acc: 1.0000 - dense_3_acc: 1.0000 - dense_4_acc: 1.0000 - de

In [7]:
board = chess.Board()

y = create_input(board)

p = model.predict(y)

interpret_pred(p)

[('e1', 'e4'), ('e1', 'e4'), ('e1', 'e4'), ('e1', 'e4'), ('e1', 'e4')]

In [0]:
## Save Model
# model.save('./drive/My Drive/Chess/simple_cnn.h5')

## Load Model
# model = load_model('./drive/My Drive/Chess/simple_cnn.h5')

In [0]:
# p2 = odel.predict(y)
# interpret_pred(p2)